# TSS annotation with CellOracle

In [1]:
import pandas as pd
import numpy as np
from celloracle import motif_analysis as ma
import celloracle as co
import mudata as mu
import os
import argparse

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


/cellar/users/aklie/opt/miniconda3/envs/test_celloracle/lib/python3.10/site-packages/loompy/bus_file.py:68: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def twobit_to_dna(twobit: int, size: int) -> str:
/cellar/users/aklie/opt/miniconda3/envs/test_celloracle/lib/python3.10/site-packages/loompy/bus_file.py:85: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def dn

In [2]:
path_data = "/cellar/users/aklie/data/datasets/neurips2021_small/annotation/2023_12_05/mdata.h5mu"
path_all_peaks = "/cellar/users/aklie/data/datasets/neurips2021_small/analysis/celloracle/2024_05_01/all_peaks.csv"
path_connections = "/cellar/users/aklie/data/datasets/neurips2021_small/analysis/celloracle/2024_05_01/cicero_connections.csv"
organism = "human"
thr_coaccess = 0.1
path_out =  "/cellar/users/aklie/data/datasets/neurips2021_small/analysis/celloracle/2024_05_01/processed_peak_file.csv"

In [3]:
# Load scATAC-seq peak list
peaks = pd.read_csv(path_all_peaks, index_col=0).x.values.astype('U')
peaks = np.char.replace(peaks, '-', '_')

In [4]:
# Load Cicero coaccessibility scores
cicero_connections = pd.read_csv(path_connections, index_col=0)
cicero_connections['Peak1'] = np.char.replace(cicero_connections['Peak1'].values.astype('U'), '-', '_')
cicero_connections['Peak2'] = np.char.replace(cicero_connections['Peak2'].values.astype('U'), '-', '_')

In [5]:
# Determine genome
if organism == 'human':
    genome = 'hg38'
elif organism == 'mouse':
    genome = 'mm10'

In [6]:
# Extract tss information
tss_annotated = ma.get_tss_info(peak_str_list=peaks, ref_genome=genome)

que bed peaks: 8192
tss peaks in que: 2227


***** WARNING: File /tmp/pybedtools.8jjvno7q.tmp has inconsistent naming convention for record:
GL000205.2	63003	63892

***** WARNING: File /tmp/pybedtools.8jjvno7q.tmp has inconsistent naming convention for record:
GL000205.2	63003	63892



In [14]:
# Integrate
integrated = ma.integrate_tss_peak_with_cicero(tss_peak=tss_annotated,
                                               cicero_connections=cicero_connections)

In [16]:
# Process
integrated = integrated[integrated['coaccess'] >= thr_coaccess]
integrated['peak_id'] = integrated['peak_id'].str.replace('_', '-')
integrated = integrated.rename(columns={'peak_id': 'cre', 'gene_short_name': 'gene', 'coaccess': 'score'})
integrated = integrated.sort_values(['cre', 'score'], ascending=[True, False])

In [17]:
# Remove unexpressed genes
genes = mu.read(os.path.join(path_data, 'rna')).var.index.values.astype('U')
integrated = integrated[integrated['gene'].isin(genes)]

In [23]:
# Log how many peaks are distal and proximal (score 1.0)
print('Number of distal peaks:', integrated[integrated['score'] < 1.0].shape[0])
print('Number of proximal peaks:', integrated[integrated['score'] == 1.0].shape[0])

Number of distal peaks: 63
Number of proximal peaks: 63


In [25]:
integrated[integrated['score'] < 1.0]

,cre,gene,score
3778,chr1-44785971-44786863,ZSWIM5,0.629032
3781,chr1-45018393-45019293,ZSWIM5,0.551528
3783,chr1-45035408-45036325,ZSWIM5,0.678244
3784,chr1-45037244-45038086,ZSWIM5,0.247496
3915,chr1-95043139-95044029,TLCD4,0.595952
...,...,...,...
4979,chr4-88592913-88593681,FAM13A,0.295680
5557,chr6-90479001-90479909,BACH2,0.519388
5847,chr8-21984926-21985818,DMTN,0.231040
5886,chr8-25171185-25172009,DOCK5,0.209174


In [26]:
integrated[integrated['score'] == 1.0]

,cre,gene,score
3109,chr1-107964928-107965436,VAV3,1.0
3467,chr1-167630023-167630677,RCSD1,1.0
3571,chr1-224433432-224434024,WDR26,1.0
3591,chr1-231626280-231627069,DISC1,1.0
3786,chr1-45206016-45206872,ZSWIM5,1.0
...,...,...,...
5887,chr8-25184219-25185134,DOCK5,1.0
5908,chr8-66712260-66713148,SGK3,1.0
5911,chr8-66774787-66775705,SGK3,1.0
6231,chrX-134796760-134797638,FAM122B,1.0


Exception ignored in atexit callback: <bound method InteractiveShell.atexit_operations of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x155551781c30>>
Traceback (most recent call last):
  File "/cellar/users/aklie/opt/miniconda3/envs/test_celloracle/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3917, in atexit_operations
    self._atexit_once()
  File "/cellar/users/aklie/opt/miniconda3/envs/test_celloracle/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3900, in _atexit_once
    self.history_manager.end_session()
  File "/cellar/users/aklie/opt/miniconda3/envs/test_celloracle/lib/python3.10/site-packages/IPython/core/history.py", line 583, in end_session
    self.writeout_cache()
  File "/cellar/users/aklie/opt/miniconda3/envs/test_celloracle/lib/python3.10/site-packages/decorator.py", line 232, in fun
    return caller(func, *(extras + args), **kw)
  File "/cellar/users/aklie/opt/miniconda3/envs/test_celloracle/lib/python3.10/site-pac

In [19]:
# Write
integrated.to_csv(path_out, index=False)

# Write command to run

```bash
python /cellar/users/aklie/opt/gene_program_evaluation/src/inference/grn_models/celloracle/workflow/scripts/r2g.py \
-d /cellar/users/aklie/data/datasets/neurips2021_small/annotation/2023_12_05/mdata.h5mu \
-a /cellar/users/aklie/data/datasets/neurips2021_small/analysis/celloracle/2024_05_01/all_peaks.csv \
-c /cellar/users/aklie/data/datasets/neurips2021_small/analysis/celloracle/2024_05_01/cicero_connections.csv \
-o human \
-t 0.1 \
-p /cellar/users/aklie/data/datasets/neurips2021_small/analysis/celloracle/2024_05_01/processed_peak_file.csv
```

# DONE!

---